In [1]:
#This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

import numpy as np
import math
from tensorflow.contrib import rnn
import tensorflow as tf
import numpy as np
from datetime import datetime

class Dataset_Manager():
    def __init__(self, dataset_split, TRAIN_VAL_SPLIT, sequence_length, batch_size):
        self.downsample = int(150000/sequence_length)
        print("Sequence lentgh of {} where the original length of 150.000 has been downsampled of: {}".format(sequence_length, self.downsample))
        train_samples = int(round(dataset_split*TRAIN_VAL_SPLIT))
        validation_samples = int(dataset_split-train_samples)
        print("The dataset contains {} train samples and {} validation samples which is a {} ratio".format(
            train_samples, validation_samples, TRAIN_VAL_SPLIT))

        index = np.arange(dataset_split)
        np.random.shuffle(index)

        self.validation_split = index[validation_samples:]
        self.train_split = index[:train_samples]
        self.seq_length = sequence_length

        self.total_batches_train = math.ceil(len(self.train_split)/batch_size)
        self.total_batches_validation = math.ceil(len(self.validation_split)/batch_size)
        self.batch_size = batch_size

    def generate_batch(self):

        np.random.shuffle(self.train_split)

        np.random.shuffle(self.validation_split)

    def get_next_batch(self, data, labels, train, step):
        minibatch = []
        minilabels = []
        if train:
            for i in range(self.batch_size):
                start_index = int(self.train_split[(self.batch_size*step)+i]*self.seq_length)
                train_batch = data[start_index:(start_index+150000):self.downsample]
                train_batch = np.expand_dims(train_batch, -1)

                # TODO: Review properly which label to use
                train_y = labels[start_index+150000-1]
                
                minibatch.append(train_batch)
                print(train_y)
                print(train_y.shape)
                print(train_batch.shape)
                print("---")
                minilabels.append(train_y)
        else:
            for i in range(self.batch_size):
                start_index = int(self.validation_split[(self.batch_size*step)+i]*self.seq_length)
                val_batch = data[start_index:(start_index+150000):self.downsample]
                val_y = labels[start_index+self.seq_length-1]
                #val_batch = np.expand_dims(val_batch, -1)
                minibatch.append(val_batch)
                minilabels.append(val_y)
        print(minilabels)
        #minilabels = np.expand_dims(minilabels, -1)
        print(minilabels)
        return minibatch, minilabels
        
class model(object):

    def __init__(self, x, seq_lenght, skip_layer, is_training, whichmodel):
        # define constants
        # unrolled through 49 time steps
        self.seq_lenght = seq_lenght

        # hidden LSTM units
        self.lstm_units = 512



        self.SKIP_LAYER = skip_layer

        self.is_training = is_training


        if whichmodel == 'LSTM':
            self.y = self.create_lstmmodel(x, is_training)
        else:
            print("Error creating the model")

    def create_lstmmodel(self,x, is_training):
        with tf.variable_scope('Single_LSTM', reuse=tf.AUTO_REUSE):
            print("\n\nShape of LSTM")
            
            print("Input X shape: ", x.get_shape())
            #print(tf.shape(x))
            #print("------")
    
    
            inputs_unstack = tf.unstack(x, axis=1)
            #print(len(inputs_unstack))
            print("Unstack shape: ", tf.shape(inputs_unstack))
            #print("------")
            fc7_out = []
            for i in inputs_unstack:
                flattened = tf.reshape(i,[-1, 2])
                fc7_out.append(flattened)
    
            aux = np.array(fc7_out)
            #print(len(fc7_out))
            #print(fc7_out[1].get_shape())
            print(aux.shape)
            #print("------")
    
    
    
            lstm_layer = rnn.BasicLSTMCell(self.lstm_units, forget_bias=1)
            lstm_outputs, _ = rnn.static_rnn(lstm_layer, fc7_out, dtype="float32")
    
            if is_training is not None:
                wd1 = tf.get_variable("wd1", [self.lstm_units, 1], initializer=tf.contrib.layers.xavier_initializer(),trainable=False)
                bd1 = tf.get_variable("bd1", [1], initializer=tf.contrib.layers.xavier_initializer(), trainable=False)
            else:
                wd1 = tf.get_variable("wd1", [self.lstm_units, 1], initializer=tf.contrib.layers.xavier_initializer(),trainable=True)
                bd1 = tf.get_variable("bd1", [1], initializer=tf.contrib.layers.xavier_initializer(), trainable=True)
                
    
            # Regression
            fc1 = tf.matmul(lstm_outputs[-1], wd1) + bd1
            print("FC1 shape: ", tf.shape(fc1))
            
            print("---- \n\n")
            #y = tf.nn.softmax(fc1)
    
            return fc1

    def load_initial_weights(self, session):

        # Load the weights into memory
        weights_dict = np.load("DEFAULT", encoding='bytes').item()

        # Loop over all layer names stored in the weights dict
        for op_name in weights_dict:

            # Check if layer should be trained from scratch
            if op_name not in self.SKIP_LAYER:

                with tf.variable_scope(op_name, reuse=True):

                    # Assign weights/biases to their corresponding tf variable
                    for data in weights_dict[op_name]:

                        # Biases
                        if len(data.shape) == 1:
                            var = tf.get_variable('biases', trainable=False)
                            session.run(var.assign(data))

                        # Weights
                        else:
                            var = tf.get_variable('weights', trainable=False)
                            session.run(var.assign(data))
                            

#Seeds
np.random.seed(1234)
tf.set_random_seed(1234)

CUDA_VISIBLE_DEVICES=0
config = tf.ConfigProto()

# Variables
TRAIN_PATH = "../input/train.csv"
TARIN_VAL_SPLIT = 0.75
filewriter_path = ""
checkpoint_path = ""

# Local variables
seq_length = 150
batch_size = 16
n_features = 1
starter_learning_rate = 0.0001
epochs = 50



print("Loading data")
train = pd.read_csv(TRAIN_PATH, dtype={'acoustic_data': np.int16, 'time_to_failure': np.float32})


print("Data with shape {} has the following type of data:".format(train.shape))



dataset_split = np.floor(train.shape[0]/seq_length) #Number of samples used on training
acoustic_data = train.acoustic_data.values
print(acoustic_data.shape)
del(train['acoustic_data'])
ttf = train.time_to_failure.values
print(ttf.shape)
del(train)

dataset = Dataset_Manager(dataset_split, TARIN_VAL_SPLIT, seq_length, batch_size)

# Model
tf.reset_default_graph()
x = tf.placeholder(tf.float32, [None, seq_length, n_features])
y = tf.placeholder(tf.float32, [None, 1])
is_training = tf.placeholder(tf.bool)

kernel = model(x, seq_length, [], is_training, 'LSTM')
score = kernel.y
print("Score shape: ", tf.shape(score))
# List of trainable variables
var_list = tf.trainable_variables()
print("----")

with tf.name_scope("cost_function"):
    print("Loading loss")
    print(tf.shape(y),tf.shape(score))
    loss = tf.losses.absolute_difference(y, score)
    tf.summary.scalar('Absolute_error', loss)

with tf.name_scope("mean_absolute_error"):
    eval_metrics_ops = tf.metrics.mean_absolute_error(y,score)
    tf.summary.scalar('MAE_metric', eval_metrics_ops)

with tf.name_scope("train"):
    # add an optimiser
    print("Loading gradients")
    global_step = tf.Variable(0, trainable=False)
    print("Global step")
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                               100, 0.95, staircase=True)
    print("learning rate")

    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
    print("Optimizer loading")
    grads = tf.gradients(loss, tf.trainable_variables())
    grads = list(zip(grads, tf.trainable_variables()))
    print("Grads")

#for var in var_list:
#        tf.summary.histogram(var.name, var)

for grad, var in grads:
    #print(var)
    #print(grad)
    #print("-----")
    tf.summary.histogram(var.name + '/gradient', grad)




# Merge all summaries together
merged_summary = tf.summary.merge_all()

# Initialize the FileWriter
writer_train = tf.summary.FileWriter(filewriter_path + '/train')
writer_test = tf.summary.FileWriter(filewriter_path + '/validation')

# Initialize a saver for store model checkpoints
saver = tf.train.Saver(save_relative_paths=True)



#with tf.Session() as sess:
with tf.Session(config = config) as sess:
    # initialise the variables
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())

    #writer_train.add_graph(sess.graph)
    #writer_test.add_graph(sess.graph)

    print("{} Start training...".format(datetime.now()))
    print("{} Open Tensorboard at --logdir {}".format(datetime.now(),
                                                     filewriter_path))
    checkpoints = 0
    for epoch in range(epochs):
        num_batches = 0
        dataset.generate_batch()
        total_batches_train = dataset.total_batches_train
        total_batches_test = dataset.total_batches_validation
        for step in range(dataset.total_batches_train):
            gs = (epoch * dataset.total_batches_train) + step + 1

            batch_x, batch_y = dataset.get_next_batch(acoustic_data, ttf , True, step)
            print("Shape X: ", np.asarray(batch_x).shape)
            batch_y = np.expand_dims(batch_y, -1)
            print("Shape Y: ", np.asarray(batch_y).shape)
            s, _, loss_mse = sess.run([merged_summary, optimizer, loss],
                               feed_dict={x: batch_x, y: batch_y, global_step: gs, is_training: True})
                               
            
            if (step % 50 == 0):
                print("MSE: ", loss_mse, "epoch ", epoch, " global step: ", gs, " training: ",
                      round(100 * float(step) / float(dataset.total_batches_train), 3), '%')
            #writer_train.add_summary(s, (epoch * num_batches) + step)

            for i in range(total_batches_test):
                if (i % 100 == 0):
                    print(" validating: ", round(100 * float(i) / float(total_batches_test), 3), '%')

                batch_test_x, batch_test_y = dataset.get_next_batch(acoustic_data, ttf, True, i)
                batch_test_y = np.expand_dims(batch_test_y, -1)
                s, mse_val, result = sess.run([merged_summary, eval_metrics_ops, score],
                                                 feed_dict={x: batch_test_x, y: batch_test_y, is_training: False})
                #writer_test.add_summary(s, (epoch * dataset.total_batches_validation) + i + step)

                print("{} Saving checkpoint of model...".format(datetime.now()))
                # save checkpoint of the model for each epoch
                checkpoint_name = os.path.join(checkpoint_path,
                                                   'model_epoch' + str(checkpoints) + '.ckpt')
                #save_path = saver.save(sess, checkpoint_name)
                checkpoints += 1

                print("{} Model checkpoint saved at {}".format(datetime.now(),checkpoint_name))
                print("{} MSE Accuracy Last = {:.4f}".format(datetime.now(), mse_val))


['sample_submission.csv', 'test', 'train.csv']


ImportError: Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/usr/local/anaconda3/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/usr/local/anaconda3/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/usr/local/anaconda3/lib/python3.6/imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "/usr/local/anaconda3/lib/python3.6/imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: libcublas.so.9.0: cannot open shared object file: No such file or directory


Failed to load the native TensorFlow runtime.

Se